In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = (SparkSession.builder.master("local[2]").appName("demo")
         .config(map={
    "spark.driver.memory": "2G",
    "spark.jars.packages": "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1,org.postgresql:postgresql:42.6.0",
    "spark.sql.extensions": "com.datastax.spark.connector.CassandraSparkExtensions",
    "spark.sql.catalog.comwatt": "com.datastax.spark.connector.datasource.CassandraCatalog",
    "spark.sql.catalog.comwatt.spark.cassandra.connection.compression": "LZ4",
    "spark.sql.catalog.comwatt.spark.cassandra.connection.host": "localhost:9042",
}).getOrCreate())

23/12/11 22:35:54 WARN Utils: Your hostname, LTO-CWT060 resolves to a loopback address: 127.0.1.1; using 192.168.0.23 instead (on interface wlp0s20f3)
23/12/11 22:35:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/chen/Documents/presentation-meetup-datascience/demo/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/chen/.ivy2/cache
The jars for the packages stored in: /home/chen/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a0490fd0-4111-4c96-ab9e-334a89f1e467;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in local-m2-cache
	found com.typesafe#config;1.4.1 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.26 in local-m2-cache
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.

In [4]:
from demo.adapters.postgres_reader import PostgresClient

pg_client = PostgresClient(spark, "jdbc:postgresql://localhost:5432/comwatt", "postgres", "demo_password")

In [11]:
from demo.adapters.postgres_reader import PostgresReader

PostgresReader(pg_client, "devices")().count()

3